In [138]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
driver=webdriver.Chrome()


In [139]:
#Practice extract from the very end page:
##Web scraping script for website :http://books.toscrape.com/index.html
def extract_data(driver,link):
    driver.get(link)
    #driver.get('http://books.toscrape.com/catalogue/rip-it-up-and-start-again_986/index.html')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    title=soup.find('h1').get_text()
    description_text=soup.find('h2', text='Product Description').find_next('p').get_text()
    price_number=soup.find('p',class_='price_color').get_text()
    availability=soup.find('p', class_='instock availability').get_text()
    UPC_number=soup.find('table',class_='table table-striped').find('td').get_text()

    outcome=[title,price_number,UPC_number,description_text,availability]
    return outcome
data=[]    
from urllib.parse import urljoin
main_page_url='http://books.toscrape.com/index.html'
driver.get(main_page_url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
main_page_div=soup.find_all('div',class_='side_categories')
##First find all category links then go to each category link to find books belonging to that category
for a in main_page_div:
    all_category_links=a.find_all('a',href=True)
    
    category_links=[]
    for each_category in all_category_links:
        each_category_link=urljoin(main_page_url,each_category['href'])
        category_links.append(each_category_link)
    #print(category_links)
    #remove the first link because its not necessary (do not show category)
    category_links.remove('http://books.toscrape.com/catalogue/category/books_1/index.html')
    #now go to each category- to search for all the books within that category:
    for i in category_links:
    #category_link='http://books.toscrape.com/catalogue/category/books/travel_2/index.html'
        just_category=i.replace('http://books.toscrape.com/catalogue/category/books/','').replace('/index.html','')
        driver.get(i)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        withinin_category=soup.select('h3')
    ##Now look for all the books link for each category
        for x in all_link_withinin_category:
            links=x.find_all('a',href=True)
    
            all_book_links=[]
    
            for link in links:
                book_link=urljoin(i,link['href'])
                all_book_links.append(book_link)

            for a in all_book_links:
                result=extract_data(driver,a)
        #print(result[1])
                data.append({'Title':result[0],'Price':result[1],'UPC':result[2],'Book Description':result[3],'Available':result[4],'Category':just_category})
        
    df=pd.DataFrame(data)
print(df)
        
        
    #get access to each 

C:\Users\Sophie\AppData\Local\Temp\ipykernel_18276\4288199038.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_text=soup.find('h2', text='Product Description').find_next('p').get_text()


                                                 Title   Price  \
0                              It's Only the Himalayas  £45.17   
1    Full Moon over Noah’s Ark: An Odyssey to Mount...  £49.43   
2    See America: A Celebration of Our National Par...  £48.87   
3    Vagabonding: An Uncommon Guide to the Art of L...  £36.94   
4                                 Under the Tuscan Sun  £37.33   
..                                                 ...     ...   
545                           The Great Railway Bazaar  £30.54   
546                   A Year in Provence (Provence #1)  £56.88   
547  The Road to Little Dribbling: Adventures of an...  £23.21   
548          Neither Here nor There: Travels in Europe  £38.95   
549                 1,000 Places to See Before You Die  £26.08   

                  UPC                                   Book Description  \
0    a22124811bfa8350  “Wherever you go, whatever you do, just . . . ...   
1    ce60436f52c5ee68  Acclaimed travel writer Rick Ant

In [153]:
##Cleaning dataset after scraping:
#Clean the category and available columns
data_copy=df.copy()
data_copy['Category']=data_copy['Category'].apply(lambda x: x.split('_')[0])
data_copy['Available']=data_copy['Available'].apply(lambda x: x.replace('\n\n    \n        ','').replace('\n    \n',''))